In [1]:
import requests
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz

c:\Users\Josvaldes\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
def obtener_info_selma_digital(nombre_producto):
    # URL de Selma Digital
    url_selma_digital = f'https://selmadigital.com/shop?search={nombre_producto}'

    # Realizar la solicitud HTTP
    response_selma_digital = requests.get(url_selma_digital)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response_selma_digital.status_code == 200:
        # Parsear el contenido HTML con BeautifulSoup
        soup_selma_digital = BeautifulSoup(response_selma_digital.content, 'html.parser')

        # Buscar los elementos que contienen el título con la clase "product-card-design8-vertical__name"
        titles = soup_selma_digital.find_all('div', class_='product-card-design8-vertical__name')

        # Buscar los elementos que contienen el precio con la clase "product-card-design8-vertical__price"
        prices = soup_selma_digital.find_all('div', class_='product-card-design8-vertical__price')

        # Crear una lista para almacenar los productos
        productos = []

        # Iterar sobre los elementos y obtener los títulos y precios
        for title, price in zip(titles, prices):
            # Obtener el título y el precio y agregarlos a la lista
            titulo = title.text.strip()
            precio = price.text.strip().replace('$', '').replace(',', '')  # Eliminar caracteres no numéricos del precio
            productos.append({'Título': titulo, 'Precio': float(precio)})

        return productos

    return None

In [4]:
def obtener_info_openfarma(nombre_producto):
    # URL de Openfarma
    url_openfarma = f'https://www.openfarma.com.ar/products?utf8=%E2%9C%93&keywords={nombre_producto}'

    # Realizar la solicitud HTTP
    response_openfarma = requests.get(url_openfarma)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response_openfarma.status_code == 200:
        # Parsear el contenido HTML con BeautifulSoup
        soup_openfarma = BeautifulSoup(response_openfarma.content, 'html.parser')

        # Buscar los elementos que contienen el título con la clase "card-title"
        titles = soup_openfarma.find_all('h2', class_='card-title')

        # Buscar los elementos que contienen el precio con la clase "promo"
        prices = soup_openfarma.find_all('span', class_='regular')

        # Crear una lista para almacenar los productos
        productos = []

        # Iterar sobre los elementos y obtener los títulos y precios
        for title, price in zip(titles, prices):
            # Obtener el título y el precio y agregarlos a la lista
            titulo = title.text.strip()
            precio = price.text.strip().replace('$', '').replace('.', '').replace(',', '.')  # Formatear el precio
            productos.append({'Título': titulo, 'Precio': float(precio)})

        return productos

    return None

In [5]:
def comparar_precios(productos_selma_digital, productos_openfarma):
    precios_selma_digital = {producto['Título']: producto['Precio'] for producto in productos_selma_digital}
    precios_openfarma = {producto['Título']: producto['Precio'] for producto in productos_openfarma}

    for titulo_selma, precio_selma in precios_selma_digital.items():
        for titulo_openfarma, precio_openfarma in precios_openfarma.items():
            similitud = fuzz.token_sort_ratio(titulo_selma, titulo_openfarma)
            if similitud >= 80:  # Umbral de similitud
                if precio_selma < precio_openfarma:
                    print(f'Selma Digital tiene un precio más bajo para "{titulo_selma}": ${precio_selma}')
                elif precio_selma > precio_openfarma:
                    print(f'Openfarma tiene un precio más bajo para "{titulo_openfarma}": ${precio_openfarma}')
                else:
                    print(f'Ambos tienen el mismo precio para "{titulo_selma}": ${precio_selma}')